# Toronto Capstone Project - Week 1
Getting neighborhood data from Wikipedia

Step 1: Data Wrangling was accomplished by cutting and pasting from Wikipedia site into Excel spreadsheet
Step 2: Uploaded Excel spreadsheet to my Capstone - Data Science Watson Project as a Data Asset.

*****  setting up tools for project

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import xlrd
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

Convert excel file to pandas df and view data for validity

Unfortunately, this method will hide code as it contains my credentials, but you'll see the DF output is what is asked for in the assignment.

In [2]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Borough,Neighborhood
0,M1S,Scarborough,Agincourt
1,M8W,Etobicoke,"Alderwood, Long Branch"
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
3,M2K,North York,Bayview Village
4,M5M,North York,"Bedford Park, Lawrence Manor East"


In [3]:
df_data_0.shape

(103, 3)

##  This is beginning of 2nd part of assignment - Getting the Longitude/Latitude coordinates for the neighborhoods

### Couldn't get Python Geocoder to run correctly so went with the CSV file route

In [4]:
filename = 'http://cocl.us/Geospatial_data'

In [5]:
df_geo = pd.read_csv(filename)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Rename columns to have consistent "key" to do merge of the 2 DFs

In [6]:
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#####  Merge the 2 DFs and create new DF for input to Foursquare for mapping and clustering for 3rd part of assignment

In [7]:
tor_geo2 = pd.merge(df_data_0, df_geo, on='PostalCode', how='outer')
tor_geo2.head(20)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,North York,Bayview Village,43.786947,-79.385975
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
7,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
8,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


In [8]:
tor_geo2.shape

(103, 5)

# Beginning of 3rd Part of assignment

Connect libraries and tools for plotting, clustering and mapping

In [9]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # transform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

#### Explore Toronto neighborhoods

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create map of Toronto with neighborhoods labeled

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tor_geo2['Latitude'], tor_geo2['Longitude'], tor_geo2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

#### Explore and cluster neighboorhoods using Foursquare API

In [38]:
client_id = 'VFND3ZHY5XKTXP5BV2FNNGH4DAX0TWPJBIL01CMAPCXMPHNO' # my Foursquare ID
client_secret = 'ZWEBHLRGZDWUDULCV15X5GV4ZU21XOEW1VZHIB40ZBNPLPER' # my Foursquare Secret
version = '20180605' # Foursquare API version


#####   Going to focus on neighborhoods in Scarborough borough for simplicity - so creating new DF of those neighborhoods

In [39]:
scar_data = tor_geo2[(tor_geo2.Borough == "Scarborough")]
scar_data.head(20)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
11,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
15,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302
16,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
24,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304
37,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
38,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
47,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
54,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353


In [40]:
scar_data.shape


(17, 5)

#####   Get the borough's map after finding geo coordinates

In [41]:
address2 = "Scarborough, ON"

geolocator = Nominatim(user_agent="scar_explorer")
location = geolocator.geocode(address2)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto Scarborough are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto Scarborough are 43.773077, -79.257774.


In [42]:
# create map of Scarborough neighborhoods
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_scarborough)
    
map_scarborough    

#### Let's explore Scarborough!!

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=VFND3ZHY5XKTXP5BV2FNNGH4DAX0TWPJBIL01CMAPCXMPHNO&client_secret=ZWEBHLRGZDWUDULCV15X5GV4ZU21XOEW1VZHIB40ZBNPLPER&v=20180604&ll=43.773077,-79.257774&radius=500&limit=100'.format(
            client_id,
            client_secret,
            version,
            lat,
            lng,
            radius,
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    return(nearby_venues)

nearby_venues.head()

NameError: name 'nearby_venues' is not defined